# Live/Design: control modes

With *pyAML* it is possible to use the same commands either:
1. live: on the real accelerator
2. design: in simulations
3. errors: in simulations with errors (and for several seeds)
4. live (with a different prefix/host): in a virtual control system (same as in point 1. for those laboratories that have a simulated control system)

Those are called "control modes" and may be used 
- at a global script level (for testing in simulations and then using in real accelerator measurements with a simple switch/comment) 
- anywhere in the script 

Using pyAML on a real accelerator

In [1]:
import os

import numpy as np

from pyaml.accelerator import Accelerator

instantiate a pyAML accelerator

In [2]:
SR: Accelerator = Accelerator.load("../../tests/config/EBSTune.yaml")

12 Feb% 2026, 12:21:35 | WARNING | PyAML Tango control system binding (0.3.3) initialized with name 'live' and TANGO_HOST=ebs-simu-3:10000


**Switch between live and design**

Access some magnet families defined in the configuration is simple. One can use sr.live for live control system or sr.design for the simulator part. They provide identical interface.

In [3]:
sr = SR.design  # simulations
# sr = SR.live # act on real accelerator

In [4]:
quadForTune = sr.get_magnets("QForTune")

tune_device = sr.get_betatron_tune_monitor("BETATRON_TUNE")

For example a tune response matrix is evaluated below. This script is used identically for computation of theoretical values or to measure the response directly on the accelerator.

In [5]:
# Build tune response matrix
initial_tune = tune_device.tune.get()
print(f"Tune via pyAML interface for design mode {initial_tune}")
tunemat = np.zeros((len(quadForTune), 2))

for idx, m in enumerate(quadForTune):
    str = m.strength.get()
    m.strength.set(str + 1e-4)
    dq = tune_device.tune.get() - initial_tune
    tunemat[idx] = dq * 1e4
    m.strength.set(str)

Tune via pyAML interface for design mode [0.15999673 0.33999769]


At any point in the code it is always possible to still access any of the control modes

In [6]:
SR.design.get_lattice().disable_6d()
tune_design = SR.design.get_lattice().get_tune()

print(f"Tune directly from the lattice {tune_design}")

Tune directly from the lattice [0.16000001 0.33999986]
